In [2]:
import requests
import os
import json
from apis_keys import twitter_api_keys
import csv
import pymysql
from datetime import datetime

import time

In [1]:
pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

     |████████████████████████████████| 205 kB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 28.4 MB/s eta 0:00:01     |████████▉                       | 1.1 MB 28.4 MB/s eta 0:00:01     |███████████████████████████▎    | 3.5 MB 28.4 MB/s eta 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.28.1
    Uninstalling google-cloud-bigquery-2.28.1:
      Successfully uninstalled google-cloud-bigquery-2.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires grpcio~=1.34.0, but you have grpcio 1.43.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "notebook-BQ-key-disco-sky-304721-761072cd7b3a.json"

## Create Dataset and Table

In [4]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

dataset_id ="disco-sky-304721.twitterradio"  #"{}.your_dataset".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

dataset.location = "europe-west3"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/disco-sky-304721/datasets?prettyPrint=false: Already Exists: Dataset disco-sky-304721:twitterradio

In [11]:

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "disco-sky-304721.twitterradio.test_data_6"

schema = [
    bigquery.SchemaField("tweet", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("tweet_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("done_reading", "BOOLEAN", mode= "REQUIRED"),
    bigquery.SchemaField("score", "FLOAT", mode = "REQUIRED"),
    bigquery.SchemaField("ingestion_time", "INTEGER", mode = "REQUIRED")
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table disco-sky-304721.twitterradio.test_data_6


In [6]:
#!pip install pymysql --user

In [27]:

# To set your enviornment variables in your terminal run the following line:
#export 'BEARER_TOKEN' = 'AAAAAAAAAAAAAAAAAAAAAGnVWAEAAAAA95wgthSdl6LZnTpOqkM130wJ7HU%3Da8H9PZ79KfkrFbgQ9gzjAJVhsi3QdrX3wJTQ4krHomZ6AiGNr3'

bearer_token = twitter_api_keys["BEARER_TOKEN"]
#save_file = open('stream_test.json', 'a')

data_dict = dict()
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "why -is:retweet -has:media -has:links lang:en"},
       # {"value": "why  -is:retweet -has:media -has:links lang:en", "tag": "twitter"},

        #{"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    

def get_stream(set):
    counter = 0
    batch_to_insert = []
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        counter +=1
        if response_line:
            json_response = json.loads(response_line)
            tweet = json_response["data"]["text"]
            tweet_id =json_response["data"]["id"]
            ingestion_time = int(round(datetime.now().timestamp(), 5)*10000)  # float(datetime.now().timestamp())
            
            
            #print(json.dumps(json_response, indent=4, sort_keys=True))
            #print(tweet)
            #print("\n\n\n")
            #self.save_file.append(json.loads(tweet))
            #save_file.write(str(print(json_response["data"]["text"]))
            

            ## process tweets
            
            row = dict()
            ## store tweet
            row["tweet"] = tweet
            row["tweet_id"] = tweet_id
            row["ingestion_time"] = ingestion_time
            row["done_reading"] = False 
            row["score"] = 0
            #print("\n","row ", "\n", row, "\n")
            
            batch_to_insert.append(row.copy())
            
            #print("\n", "batch_to_insert", "\n", batch_to_insert, "\n")
            
            if counter == 2:
                
                counter = 0 
                errors = client.insert_rows_json(table_id, batch_to_insert)  # Make an API request.
                if errors == []:
                    print("New rows have been added.")
                else:
                    print("Encountered errors while inserting rows: {}".format(errors))

                print("\n\n","SAVED BATCH", "\n", batch_to_insert)
                batch_to_insert = []
                time.sleep(5)
            
def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)


if __name__ == "__main__":
    main()

{"data": [{"id": "1474033943330672646", "value": "why -is:retweet -has:media -has:links lang:en"}], "meta": {"sent": "2021-12-23T15:26:23.402Z", "result_count": 1}}
{"meta": {"sent": "2021-12-23T15:26:24.811Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "why -is:retweet -has:media -has:links lang:en", "id": "1474038691685453825"}], "meta": {"sent": "2021-12-23T15:26:26.459Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200
New rows have been added.


 SAVED BATCH 
 [{'tweet': 'why you so cuteee 🥺', 'tweet_id': '1474038671015772161', 'ingestion_time': 16402731906840, 'done_reading': False, 'score': 0}, {'tweet': 'Why do my folks wake up at the crack of dawn talking LOUD?!? 😭\n\nIs it just me?  I enjoy silence til noon at least', 'tweet_id': '1474038670684573701', 'ingestion_time': 16402731906841, 'done_reading': False, 'score': 0}]
New rows have been added.


 SAVED BATCH 
 [{'tweet': '@KylaInTheBurgh And to top it all off, he’s dead fr

KeyboardInterrupt: 

In [21]:
int(round(1640166302.019928, 5)*10000)

16401663020199

In [40]:
import pickle
dictionary_data = {"a": 1, "b": 2}


a_file = open("data.pkl", "wb")

pickle.dump(dictionary_data, a_file)

a_file.close()


a_file = open("data.pkl", "rb")

output = pickle.load(a_file)

print(output)

{'a': 1, 'b': 2}


In [41]:
a_file

<_io.BufferedReader name='data.pkl'>

In [42]:
b_file = open("data.pkl", "a")


In [43]:
dictionary_data = {"c": 1, "d": 2}


In [46]:
b_file.write(dictionary_data)

TypeError: write() argument must be str, not dict

In [5]:
!pip install TTS[tf] --user


  Using cached numba-0.53.0-cp37-cp37m-manylinux2014_x86_64.whl (3.4 MB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
     |████████████████████████████████| 454.3 MB 9.1 kB/s eta 0:00:011    |███                             | 42.6 MB 25.8 MB/s eta 0:00:16     |████▋                           | 65.7 MB 25.8 MB/s eta 0:00:16     |█████▌                          | 78.5 MB 25.8 MB/s eta 0:00:15     |████████▏                       | 116.2 MB 13.4 MB/s eta 0:00:26     |████████▌                       | 120.4 MB 13.4 MB/s eta 0:00:25     |█████████                       | 126.9 MB 51.0 MB/s eta 0:00:07     |█████████                       | 128.9 MB 51.0 MB/s eta 0:00:07     |███████████▎                    | 159.6 MB 51.0 MB/s eta 0:00:06     |███████████▉                    | 167.6 MB 28.8 MB/s eta 0:00:10     |████████████▎                   | 175.1 MB 28.8 MB/s eta 0:00:10     |████████████▊                   | 180.0 MB 28.8 MB/s eta 0:00:10     |███████████